Create Azure SQL Database
==================================

Description
-----------
This notebook will help you get created single Azure SQL Database inside Azure SQL Server and configure a server-level firewall rule. For more information see <a href=https://docs.microsoft.com/en-us/azure/azure-sql/database/scripts/create-and-configure-database-powershell>Use PowerShell to create a single database and configure a server-level firewall rule</a>


Steps of this procedure include:
1. Connect to Azure subscription
1. Provision resource group for SQL Managed Instance
2. Create Sql Server
3. Provision firewall rules to allow access
4. Create Sql Database

In [ ]:
# Resource Group 
$resourceGroupName  =   ""                # Name of the resource group to create in the current subscription
$location           =   ""                # Name of location (see Appendix for a list of location settings)

# SQL Server
$serverName = "mysqlserver-$(Get-Random)"
$adminLogin = ""                          # Username of Sql server. Default would be 'azureuser'
$password = ""                            # Desired password for Sql Server

# SQL Database
$databaseName = ""                        # Name of database to be created

# The ip address range that you want to allow to access your server
$startIp = "0.0.0.0"
$endIp = "0.0.0.0"

### <b>Connect to Azure </b>
Below command will open a _Dialouge Box_ asking your account credentials.

In [ ]:
Connect-AzAccount

### <b>Get Subscription</b>
Below command will open a _Dialouge Box_ with list of subscriptions. Selecting one of those will set that subscription for rest of the commands.

In [ ]:
$subscription = Get-AzSubscription | Out-GridView -PassThru
Set-AzContext -SubscriptionName $subscription

### <b>Create the New Resource Group</b>
_New-AzResourceGroup_ command will create new resource group in given subscription.

In [ ]:
# Create Azure resource group, if necessary
$rg = Get-AzResourceGroup | Where ResourceGroupName -eq $resourceGroupName

if (!$rg)
{
    # Need to create a new resource group
    Write-Output "Resource Group $resourceGroupName does not exist. Creating..."
    $rg = New-AzResourceGroup -Name $resourceGroupName -Location $location
}

### <b>Create a SQL server with a system wide unique server name</b>
_New-AzSqlServer_ will create Sql Server with required configurations.

In [ ]:
Write-Output "Creating SqlServer with name $serverName ...."
New-AzSqlServer -ResourceGroupName $resourceGroupName `
   -ServerName $serverName `
   -Location $location `
   -SqlAdministratorCredentials $(New-Object -TypeName System.Management.Automation.PSCredential `
   -ArgumentList $adminLogin, $(ConvertTo-SecureString -String $password -AsPlainText -Force))

### <b>Create a server firewall rule that allows access from the specified IP range</b>
_New-AzSqlServerFirewallRule_ creates a new firewall rule for sql server

In [ ]:
Write-Output "Configuring firewall for Sql Server..."
New-AzSqlServerFirewallRule -ResourceGroupName $resourceGroupName `
   -ServerName $ServerName `
   -FirewallRuleName "AllowedIPs" -StartIpAddress $startIp -EndIpAddress $endIp

### <b>Create SQL Database</b>
_New-AzSqlDatabase_ command will create new database in the server.

In [ ]:
Write-Output "Creating a gen5 2 vCore database..."
$database = New-AzSqlDatabase  -ResourceGroupName $resourceGroupName `
   -ServerName $ServerName `
   -DatabaseName $databaseName `
   -Edition GeneralPurpose `
   -VCore 2 `
   -ComputeGeneration Gen5 `
   -MinimumCapacity 2

### <b> Get Database details </b>
_Get-AzSqlDatabase_ command gets database details on the server specified.

In [ ]:
Get-AzSqlDatabase -ResourceGroupName $resourceGroupName -ServerName $serverName -DatabaseName $databaseName